In [143]:
import pandas as pd
import numpy as np
import datetime as dt

In [144]:
cnt=snakemake.params.countries
year=snakemake.wildcards.year

In [145]:
hdd_annual=pd.read_csv(snakemake.input.hdd_annual_totals).set_index("year").loc[:,cnt]
ued_annual=pd.read_csv(snakemake.input.ued_annual,skiprows=1).set_index("ISO2").loc[cnt,:]
cooling=pd.read_csv(snakemake.input.cooling_annual).loc[:,["Country","tot_cool"]]
heat_db=pd.read_excel(snakemake.input.heat_db,sheet_name="h&c",engine="calamine").query("Scenario == 'HRE'")
profiles=pd.read_csv(snakemake.input.heatpump_profiles,skiprows=1)
utc=pd.read_csv(snakemake.input.time_zones).set_index('ISO2').loc[cnt,"UTC"]

In [146]:
def ashp_cop(deltat):
    
    # from Iain's paper
    
    return 6.81-0.121*deltat+0.000630*deltat**2

In [147]:
def gshp_cop(deltat):
    
    # from Iain's paper
        
    return 8.77-0.150*deltat+0.000734*deltat**2

In [148]:
select=cnt+[c+"_cdh" for c in cnt]+[c+"_hdh" for c in cnt]

t2m=pd.read_csv(snakemake.input.degree_days,index_col=0).loc[:,select]
t2m.index=pd.to_datetime(t2m.index)
t2m=t2m.resample("D").mean()

In [149]:
ued_annual=ued_annual.merge(cooling,left_index=True,right_on="Country")

In [150]:
# only add cooling electricity demand that was previously removed - the removal process misses a chunk of space cooling

ued_annual["res_sc"]=(ued_annual["res_sc_elc"]/(ued_annual["res_sc_elc"]+ued_annual["ser_sc_elc"]))*ued_annual["tot_cool"]
ued_annual["ser_sc"]=(ued_annual["ser_sc_elc"]/(ued_annual["res_sc_elc"]+ued_annual["ser_sc_elc"]))*ued_annual["tot_cool"]

In [151]:
# annual useful space heat demand is adjusted up/down inputd on comparison to 2012
    
ued_annual["res_sh"]=((hdd_annual.loc[hdd_annual.index==int(year),:]
                        .values/hdd_annual.loc[hdd_annual.index==2012,:].values)
                        .reshape(-1,)*ued_annual["res_sh"])

In [152]:
ued_annual

,Country,Country_x,res_sh,res_sc,res_hw,ser_sh,ser_sc,ser_hw,res_sh_elc,res_sc_elc,ser_sh_elc,ser_sc_elc,Country_y,tot_cool
0,AT,Austria,41.4,0.01906,4.2,12.3,0.152478,2.0,3.3,0.1,0.3,0.8,AT,0.171537
5,DE,Germany,361.9,0.03748,50.4,133.8,0.262360,19.5,8.8,0.8,3.7,5.6,DE,0.299840


In [153]:
index=["Country","res_sh","res_sc","res_hw","ser_sh","ser_sc","ser_hw"]
ued_annual=pd.pivot_table(ued_annual.loc[:,index],columns="Country")

In [154]:
heat=t2m.loc[:,[c+"_hdh" for c in cnt]]/t2m.loc[:,[c+"_hdh" for c in cnt]].sum(axis=0)
heat.rename({c+"_hdh":c for c in cnt},axis=1,inplace=True)
    
cool=t2m.loc[:,[c+"_cdh" for c in cnt]]/t2m.loc[:,[c+"_cdh" for c in cnt]].sum(axis=0)
cool.rename({c+"_cdh":c for c in cnt},axis=1,inplace=True)
    
    
flat=heat*0+(1/heat.shape[0])
t2m=t2m.loc[:,cnt]
    
enduse={"res_sh":heat,"res_sc":cool,"res_hw":flat,"ser_sh":heat,"ser_sc":cool,"ser_hw":flat}

l=[]

In [155]:
for eu in enduse:
    
    d=ued_annual.loc[ued_annual.index==eu,:] \
    *heat_db.loc[heat_db["EU"]==eu,"growth"].values[0]*heat_db.loc[heat_db["EU"]==eu,"share"].values[0]

    if "sh" or "hw" in eu:
        
        # target temp assumed to be 50 degrees for sh and hw
        
        cop=ashp_cop(50.-t2m)
        
    elif "sc" in eu:
        
        # sc is alreadying electricity terms
        
        cop=1

    #if "hw" in eu:
    #    temp=enduse[eu].copy().mul(d.values,axis=1).div(cop)*1E6
        
    #else:
        
    temp=enduse[eu].copy().mul(d.values,axis=1).div(cop)*1E6
    
    if eu == "ser_sh":
        
        # scaling factors are from the when2heat commercial profiles including day of week effects
        
        temp.loc[temp.index.dayofweek > 4,:]=temp*0.93
        temp.loc[temp.index.dayofweek <= 4,:]=temp*1.03           

    out=[]
    for date,d in temp.iterrows():
        
        if eu == "ser_sh" and date.dayofweek > 4:
            eu="ser_sh_we"
        elif eu == "ser_sh" and date.dayofweek <= 4:
            eu="ser_sh_wd"
        elif eu == "ser_hw" and date.dayofweek > 4:
            eu="ser_hw_we"
        elif eu == "ser_hw" and date.dayofweek <= 4:
            eu="ser_hw_wd"
        
        prof=np.tile(profiles[eu].values.reshape(-1,1),reps=(1,d.shape[0]))
        
        if date < pd.Timestamp(date.year,3,25) or date > pd.Timestamp(date.year,10,25):
            
            # shift usage profile to align with activity - UTC reference frame
            
            for i,u in enumerate(utc*-1):
                prof[:,i]=np.roll(prof[:,i],u)
                
            out.append(d.values.reshape(1,-1)*prof)
        else:
            
            # shift usage profile to align with daylight savings activity - UTC reference frame
  
            for i,u in enumerate(utc*-1):
                prof[:,i]=np.roll(prof[:,i],u-1)
            
            out.append(d.values.reshape(1,-1)*prof)

    out=pd.DataFrame(np.concatenate(out,axis=0),
                         index=pd.date_range(start=t2m.index[0],end=t2m.index[-1]+dt.timedelta(hours=23),freq="1h"),
                         columns=temp.columns)
    l.append(out)

res_sh
res_sc
res_hw
ser_sh
ser_sc
ser_hw


In [156]:
l=pd.concat(l,axis=1)

# all this transposing looks ugly but since axis is being deprecated in groupby it is necessary

l=l.T.groupby(by=l.T.index).sum().T

In [ ]:
l.to_csv(snakemake.output[0])